# Setup 4: Widening the playing field

In [57]:
import numpy as np
import matplotlib.pyplot as plt
import imageio.v2 as imageio
import seaborn as sns
import os
from IPython.display import HTML
from time import time

In [58]:
def reward_struct(state, action, weight: int = 1, width: int = 2):
    # This is the reward structure introduced in Setup 2
    return weight  - (abs(state - action) / width)

In [59]:
def stimgen(n: int) -> float:
    # Gaussian with FWHM of 2.
    # return np.exp(- (n**2) / (4 / np.log(2)))
    # Gaussian with a FWHM of 5
    return np.exp(- (n**2) / (25 / np.log(2)))

In [60]:
class World:
    def __init__(self, n_states: int, 
                 n_signals: int, n_actions: int, 
                 seed: int = 0, reward = (1, 2)) -> None:
        self.setup = (n_signals, n_actions)
        self.n_states = n_states
        self.state = 0
        self.random = np.random.RandomState(seed)
        self.reward = reward

    def get_state(self) -> int:
        self.state = self.random.randint(self.n_states)
        return self.state

    def evaluate(self, action: int) -> int:
        # this assumes self.n_actions == self.n_states
        if self.n_states > 2:
            # circle game
            left = right = action
            for i in range(self.n_states):
                # print(i, left, right)
                if left == self.state or right == self.state:
                    return 1 - (i / 2)
                # move left pointer
                left -= 1 
                if left - 1 < 0:
                    # modulo
                    left = self.n_states - 1
                # move right pointer
                right += 1
                if right >= self.n_states:
                    # modulo
                    right = 0
        else:
            # weight minus ((shortest distance between state and action) / width)
            return reward_struct(self.state, action, weight=self.reward[0], width=self.reward[1])

In [61]:
class Sender:
    def __init__(self, n_stimuli: int, n_signals: int,
                  q_not: float = 1e-6, stimgen: bool = False) -> None:
        # n_stimuli: number of possible states in the world,
        #            each corresponding to a stimulus
        # n_signals: number of signals that can be sent in response,
        #            usually equal to the number of states in the world
        # q_not:     initial signal propensity values. Final value of null signal.
        self.n_signals = n_signals + 1 # +1 here represents null signal.    
        self.signal_weights = np.zeros((self.n_signals, n_stimuli))
        self.rew_hist = np.zeros_like(self.signal_weights)
        self.signal_weights.fill(q_not)
        self.last_situation = (0, 0)
        self.stimgen = stimgen

    def get_signal(self, stimulus: int) -> int:
        # exponential calculation
        num = np.exp(self.signal_weights[:, stimulus])
        den = np.sum(np.exp(self.signal_weights[:, stimulus]))
        probabilities = num / den
        signal = np.random.choice(self.n_signals, p=probabilities)
        if signal == self.n_signals - 1:
            return -1
        self.last_situation = (stimulus, signal)
        return signal

    def update(self, reward: int) -> None:
        # I am capping weight values at 308 due to overflow errors.
        # They must similarly be floored at -323.
        stimulus, signal = self.last_situation
        q_last = self.signal_weights[signal, stimulus]
        self.signal_weights[signal, stimulus] = max(-323, min(q_last + reward, 308))
        self.rew_hist[signal, stimulus] += reward
        # stimulus generalization
        if self.stimgen:
            l = r = stimulus
            for i in range(1,4):
                re = reward * stimgen(i)
                # reward right
                r += 1
                if r >= self.n_signals:
                    r = 0
                q_last = self.signal_weights[signal, r]
                self.signal_weights[signal, r] = max(-323, min(q_last + re, 308))
                self.rew_hist[signal, r] += re

                # reward left
                l -= 1
                if l < 0:
                    l = self.n_signals - 1
                q_last = self.signal_weights[signal, l]
                self.signal_weights[signal, l] = max(-323, min(q_last + re, 308))
                self.rew_hist[signal, r] += re

    def checkpoint(self):
        self.rew_hist /= 100
        sav = np.copy(self.rew_hist)
        self.rew_hist = np.zeros_like(self.signal_weights)
        return sav

In [62]:
class Receiver:
    def __init__(self, n_signals, n_actions, 
                 q_not: float = 1e-6, stimgen: bool = False) -> None:
        # n_signals: number of signals that can be sent in response,
        #            usually equal to the number of states in the world
        # n_actions: number of actions that can be taken in response,
        #            usually equal to the number of states in the world
        # q_not:     initial action propensity value
        self.n_actions = n_actions
        self.action_weights = np.zeros((n_signals, n_actions))
        self.rew_hist = np.zeros_like(self.action_weights)
        self.action_weights.fill(q_not)
        self.last_situation = (0, 0)
        self.stimgen = stimgen

    def get_action(self, signal: int) -> int:
        # exponential calculation
        num = np.exp(self.action_weights[signal, :])
        den = np.sum(np.exp(self.action_weights[signal, :]))
        probabilities = num / den
        action = np.random.choice(self.n_actions, p=probabilities)
        self.last_situation = (signal, action)

        return action
    
    def update(self, reward: int) -> None:
        signal, action = self.last_situation
        q_last = self.action_weights[signal, action]
        self.action_weights[signal, action] = max(-323, min(q_last + reward, 308))
        self.rew_hist[signal, action] += reward

        # stimulus generalization
        if self.stimgen:
            l = r = action
            for i in range(1,4):
                re = reward * stimgen(i)
                # reward right
                r += 1
                if r >= self.n_actions:
                    r = 0
                q_last = self.action_weights[signal, r]
                self.action_weights[signal, r] = max(-323, min(q_last + re, 308))
                self.rew_hist[signal, r] += re

                # reward left
                l -= 1
                if l < 0:
                    l = self.n_actions - 1
                q_last = self.action_weights[signal, l]
                self.action_weights[signal, l] = max(-323, min(q_last + re, 308))
                self.rew_hist[signal, r] = re

    def checkpoint(self):
        self.rew_hist /= 100
        sav = np.copy(self.rew_hist)
        self.rew_hist = np.zeros_like(self.action_weights)
        return sav

In [63]:
class History:
    def __init__(self, epochs, states, signals, actions):
        self.send_hist = np.zeros((epochs // 25, signals+1, states))
        self.reci_hist = np.zeros((epochs // 25, signals, actions))
        self.send_rew = np.zeros_like(self.send_hist)
        self.reci_rew = np.zeros_like(self.reci_hist)
        self.slows = np.zeros((epochs//100))
        self.epochs = epochs
        self.ep = 0
        self.ep2 = 0

    def add_25(self, send_weights, reci_weights):
        self.send_hist[self.ep] = send_weights
        self.reci_hist[self.ep] = reci_weights
        self.ep += 1

    def add_100(self, send_rew, reci_rew, slow):
        self.send_rew[self.ep2] = send_rew
        self.reci_rew[self.ep2] = reci_rew
        self.slows[self.ep2] = slow
        self.ep2 += 1

    def make_gif(self, fps, seed, filename_base, html=False):
        if not os.path.exists(f'./images'):
            os.mkdir(f'images') 
        for i in range(self.ep):
            fig, axs = plt.subplots(2, 1, figsize=(8, 6))
            plt.tight_layout(pad=3)

            sns.heatmap(
                np.exp(self.send_hist[i]) /
                np.exp(self.send_hist[i]).sum(axis=0),
                linewidth=0.5, linecolor='white',
                square=True, cbar=False, annot=True, fmt='.1f', ax=axs[0])
            axs[0].set_ylabel('messages')
            axs[0].set_xlabel('world states')
            axs[0].set_title(f'Sender\'s weights')

            sns.heatmap(
                (np.exp(self.reci_hist[i].T) /
                np.exp(self.reci_hist[i].T).sum(axis=0)).T,
                linewidth=0.5, linecolor='white',
                square=True, cbar=False, annot=True, fmt='.1f', ax=axs[1])
            axs[1].set_xlabel('actions')
            axs[1].set_ylabel('messages')
            axs[1].set_title(f'Receiver\'s weights')
            
            
            fig.suptitle(f'Rollout {i*25}')
            plt.savefig(f"./images/game_{i*25}.png")
            plt.close(fig)

        images = []
        for filename in [f'./images/game_{j*25}.png' for j in range(self.ep)]:
            images.append(imageio.imread(filename))
        imageio.mimsave(f'{filename_base}.gif', images, fps=fps)
        if html:
            display(HTML('<img src="{}">'.format(f'{filename_base}.gif')))
        # no return
    
    def print_send_map(self):
        final = (self.send_hist[self.ep-1] == self.send_hist[self.ep-1].max(axis=0)[None, :]).astype(int)
        print('a|s', end='')
        for i in range(final.shape[1]):
            print(f'{i:2}', end=' ')
        print()
        for i in range(final.shape[0]):
            for j in range(-1, final.shape[1]):
                if j == -1:
                    print(f'{i:2}', end=' ')
                else:
                    print(f'{final[i, j]:2}', end=' ')
            print()

    def print_reci_map(self):
        final = (self.reci_hist[-1].T == self.reci_hist[-1].T.max(axis=0)[None, :]).astype(int).T
        print('m|a', end='')
        for i in range(final.shape[1]):
            print(f'{i:2}', end=' ')
        print()
        for i in range(final.shape[0]):
            for j in range(-1, final.shape[1]):
                if j == -1:
                    print(f'{i:2}', end=' ')
                else:
                    print(f'{final[i, j]:2}', end=' ')
            print()

## Experiment

In [64]:
# 40 25-state regions
st = 1000
si = 40
ac = 1000
initial = 1
epochs = 10000

gif_fps = 15
seed = 0

reward_weight = 0.1
reward_dist = 20
rew = (reward_weight, reward_dist)

In [65]:
W = World(st, si, ac, seed, rew)
S = Sender(st, si, initial, stimgen=True)
R = Receiver(si, ac, initial, stimgen=True)
H = History(epochs, st, si, ac)

In [66]:
slow = past_rewards = 0

for epoch in range(epochs):
    stimulus = W.get_state()
    signal = S.get_signal(stimulus)
    if signal != -1:
        action = R.get_action(signal)
        reward = W.evaluate(action)
        past_rewards += reward
        S.update(reward)
        R.update(reward)
    # else null action
    
    if epoch % 25 == 0:
        # save history
        H.add_25(S.signal_weights, R.action_weights)
        

    if epoch % 100 == 0:
        slow = past_rewards / 100
        past_rewards = 0
        H.add_100(S.checkpoint(), R.checkpoint(), slow)

In [67]:
# final action-state mapping
H.print_send_map()

a|s 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 2

In [68]:
# final message-action mapping
H.print_reci_map()

m|a 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 2

In [69]:
#H.make_gif(gif_fps, seed, f'./images/{st}-{si}-{ac}_game', html=True)